In [ ]:
# !pip install langchain_community langchain_groq langchain_openai -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.9/62.9 kB 1.6 MB/s eta 0:00:00


## Single tool: Tavily Search

In [ ]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate
import os
from google.colab import userdata
os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")

tools = [TavilySearchResults(max_results=1)]

In [ ]:
import os
from langchain_groq import ChatGroq


from google.colab import userdata
os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")

# To create llm object, make sure you have 'GROQ_API_KEY' in your environment variable
# comment above line of code where we are fetching colab userdata.

llm = ChatGroq(temperature=0.1, model_name="llama-3.1-8b-instant")

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Make sure to use the tavily_search_results_json tool for information.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

# Construct the Tools agent
agent = create_tool_calling_agent(llm, tools, prompt)

In [ ]:
for tool in tools:
  print(tool.name)
  print(tool.description)
  print(tool.args)

tavily_search_results_json
A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.
{'query': {'description': 'search query to look up', 'title': 'Query', 'type': 'string'}}


In [ ]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
response = agent_executor.invoke({"input": "when did virat kohli retired from test cricket?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'Virat Kohli retirement test cricket date'}`


[{'title': 'Virat Kohli Retirement Live Updates: World cricket pays tribute as ...', 'url': 'https://timesofindia.indiatimes.com/sports/cricket/virat-kohli-test-retirement-live-update-bcci-cricket-players-fan-reaction-latest-updates/liveblog/121101473.cms', 'content': 'Virat Kohli announced his retirement from Test cricket on Monday, bringing an end to a remarkable 14-year journey in the longest format of the game. The announcement comes just ahead of India’s five-match Test series against England, set to begin on June 20 at Headingley, marking the beginning of a new World Test Championship cycle for the Indian team.In the statement announcing the retirement, Kohli wrote, "It’s been 14 years since I first wore the baggy blue in Test cricket. Honestly, I never [...] Virat Kohli, who led India to the final of the inaugural World Test Championship 

In [ ]:
response['output']

'Based on the search results, Virat Kohli announced his retirement from Test cricket on May 12, 2023.'

## Multiple tools : weather + calculator

In [ ]:
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor
from langchain_core.tools import Tool
from langchain import hub

# Define tools
def get_weather(location):
    return f"It's currently sunny and 75°F in {location}."

def calculate(expression):
    try:
        return str(eval(expression))
    except Exception as e:
        return f"Error: {str(e)}"

tools = [
    Tool.from_function(
        name="Weather",
        description="Get the current weather for a location.",
        func=get_weather
    ),
    Tool.from_function(
        name="Calculator",
        description="Calculate mathematical expressions.",
        func=calculate
    )
]


prompt = hub.pull("hwchase17/openai-functions-agent")

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [ ]:
prompt.messages[0].prompt

PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant')

In [ ]:
from langchain_openai import ChatOpenAI


llm_openai = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, api_key=userdata.get("OPENAI_API_KEY"))



agent = create_openai_functions_agent(llm_openai, tools, prompt)

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True
)

In [ ]:
# Test the agent
response = agent_executor.invoke({
    "input": "What's the weather in New York? Also, what's 25 * 16?, What's the weather at Jaipur?"
})
print(response["output"])



> Entering new AgentExecutor chain...

Invoking: `Weather` with `New York`


It's currently sunny and 75°F in New York.
Invoking: `Calculator` with `25 * 16`


400
Invoking: `Weather` with `Jaipur`


It's currently sunny and 75°F in Jaipur.The weather in New York is currently sunny and 75°F. The result of 25 * 16 is 400. The weather in Jaipur is also sunny and 75°F.

> Finished chain.
The weather in New York is currently sunny and 75°F. The result of 25 * 16 is 400. The weather in Jaipur is also sunny and 75°F.


In [ ]:
response

{'input': "What's the weather in New York? Also, what's 25 * 16?, What's the weather at Jaipur?",
 'output': 'The weather in New York is currently sunny and 75°F. The result of 25 * 16 is 400. The weather in Jaipur is also sunny and 75°F.'}

In [ ]:
# ReAct Agent